In [1]:
import persistence_homology as ph

In [2]:
import csv
import os
import json
from pathlib import Path
import logging

In [3]:
log_path = Path("notebook_demo.log")
logging.basicConfig(
    filename='automation.log',         # log file path
    filemode='a',                  # 'w' to overwrite each run, 'a' to append
    format='%(asctime)s %(levelname)-8s %(name)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO             # minimum level to record
)
logger = logging.getLogger(__name__)

logger.info("Notebook logging started")

In [4]:
def read_csv(path):
    with open(path, newline="") as f:
        reader = csv.reader(f)
        output = []
        for row in reader:
            values = []
            for value in row:
                if len(row) == 3:
                    values.append(float(value))
                elif len(row) == 2:
                    values.append(int(value))
            output.append(values)
        return output
def get_subfolders(path):
    """
    Return a list of names of all subdirectories in the given path.
    """
    return [
        name for name in os.listdir(path)
        if os.path.isdir(os.path.join(path, name))
    ]
def write_json(json_file_name, data):
    output_path = Path("{}.json".format(json_file_name))
    with output_path.open("w", encoding="utf-8") as f:
        json.dump(
            data,            # Python object
            f,               # file handle
            ensure_ascii=False,  # keep unicode chars if any
            indent=4             # pretty-print with 4-space indent
        )

In [5]:
segmentation_folder_path = "./lung_segmentations"
lung_segmentations = get_subfolders(segmentation_folder_path)

In [6]:
print(lung_segmentations)

['Lung segmentation-AI-19', 'Lung segmentation-AI-2', 'Lung segmentation-25', 'Lung segmentation-20', 'Lung segmentation-AI-32', 'Lung segmentation-52', 'Lung segmentation-48', 'Lung segmentation-AI-10', 'Lung segmentation-AI-25', 'Lung segmentation-AI-17', 'Lung segmentation-AI-15', 'Lung segmentation-AI-9', 'Lung segmentation-AI-20', 'Lung segmentation-36', 'Lung segmentation-17', 'Lung segmentation-2', 'Lung segmentation-34', 'Lung segmentation-AI-34', 'Lung segmentation-AI-29', 'Lung segmentation-19', 'Lung segmentation-65', 'Lung segmentation-AI-48', 'Lung segmentation-56', 'Lung segmentation-32', 'Lung segmentation-39', 'Lung segmentation-4', 'Lung segmentation-57', 'Lung segmentation-9', 'Lung segmentation-AI-39', 'Lung segmentation-10', 'Lung segmentation-15', 'Lung segmentation-AI-36', 'Lung segmentation-AI-4']


# Filtration Algorithm Start

In [7]:
for lung_segmentation in lung_segmentations:
    print(lung_segmentation)

Lung segmentation-AI-19
Lung segmentation-AI-2
Lung segmentation-25
Lung segmentation-20
Lung segmentation-AI-32
Lung segmentation-52
Lung segmentation-48
Lung segmentation-AI-10
Lung segmentation-AI-25
Lung segmentation-AI-17
Lung segmentation-AI-15
Lung segmentation-AI-9
Lung segmentation-AI-20
Lung segmentation-36
Lung segmentation-17
Lung segmentation-2
Lung segmentation-34
Lung segmentation-AI-34
Lung segmentation-AI-29
Lung segmentation-19
Lung segmentation-65
Lung segmentation-AI-48
Lung segmentation-56
Lung segmentation-32
Lung segmentation-39
Lung segmentation-4
Lung segmentation-57
Lung segmentation-9
Lung segmentation-AI-39
Lung segmentation-10
Lung segmentation-15
Lung segmentation-AI-36
Lung segmentation-AI-4


In [8]:
for lung_segmentation in lung_segmentations:
    folder_path = './lung_segmentations/{}'.format(lung_segmentation)
    edges_path = '/edges.csv'
    vertices_path = '/vertices.csv'
    vertices = read_csv(folder_path + vertices_path)
    edges = read_csv(folder_path + edges_path)
    print(len(edges))

123279
138334
67500
45903
55437
103843
108681
77781
67551
119766
79816
73461
45903
98091
119766
138334
92825
92825
63121
123279
85228
108234
50826
55377
78654
78781
84330
74136
78654
77781
79816
98091
78781


In [9]:
json_data = {}
for lung_segmentation in lung_segmentations:
    folder_path = './lung_segmentations/{}'.format(lung_segmentation)
    edges_path = '/edges.csv'
    vertices_path = '/vertices.csv'
    vertices = read_csv(folder_path + vertices_path)
    edges = read_csv(folder_path + edges_path)
    directions = ph.generate_sphere_points(20, 5, 0.0000001)
    #directions = [[0,0,1]]
    print("Lung Segmentation-{}:".format(lung_segmentation))
    segmentation_data = []
    for direction in directions:
        bz = ph.BettiZero(direction, vertices, edges)
        components, mergers, verts, births = bz.compute_persistence()
        intervals = ph.compute_intervals(births, mergers)
        largest_bar = ph.compute_largest_bar(intervals)
        data = {
            "direction": direction,
            "intervals": intervals,
            "largest_bar": largest_bar,
            "components": list(components)
        }
        segmentation_data.append(data)
        print("Direction: {}. Largest Bar: {}.".format(direction, largest_bar))
        logger.info(data)
    json_data[lung_segmentation] = segmentation_data
write_json("BettiZeroSegmentations", json_data)

Lung Segmentation-Lung segmentation-AI-19:
Direction: [-1.  0.  0.]. Largest Bar: (np.float64(39.0), [np.float64(-353.5), np.float64(-314.5)]).
Direction: [-0.95105652 -0.30901699  0.        ]. Largest Bar: (np.float64(31.837915231136947), [np.float64(-413.1437101096987), np.float64(-381.30579487856176)]).
Direction: [-0.95105652  0.30901699  0.        ]. Largest Bar: (np.float64(47.4327981489061), [np.float64(-264.0325350256209), np.float64(-216.59973687671481)]).
Direction: [-0.80901699 -0.58778525  0.        ]. Largest Bar: (np.float64(21.55931349165087), [np.float64(-432.34603533236975), np.float64(-410.7867218407189)]).
Direction: [-0.80901699  0.          0.58778525]. Largest Bar: (np.float64(49.55177385960718), [np.float64(-195.83513216871307), np.float64(-146.2833583091059)]).
Direction: [-0.80901699  0.         -0.58778525]. Largest Bar: (np.float64(58.26221995456092), [np.float64(-398.25449069940623), np.float64(-339.9922707448453)]).
Direction: [-0.80901699  0.58778525  0.  

TypeError: Object of type ndarray is not JSON serializable

In [18]:
print(json_data['Lung segmentation-AI-19'])

[{'direction': array([-1.,  0.,  0.]), 'intervals': [[np.float64(-359.5), -1], [np.float64(-357.5), np.float64(-332.5)], [np.float64(-356.5), np.float64(-352.5)], [np.float64(-356.5), np.float64(-346.5)], [np.float64(-353.5), np.float64(-314.5)], [np.float64(-352.5), np.float64(-330.5)], [np.float64(-351.5), -1], [np.float64(-350.5), np.float64(-342.5)], [np.float64(-349.5), np.float64(-337.5)], [np.float64(-347.5), np.float64(-346.5)], [np.float64(-347.5), -1], [np.float64(-346.5), np.float64(-335.5)], [np.float64(-344.5), np.float64(-343.5)], [np.float64(-343.5), np.float64(-335.5)], [np.float64(-340.5), np.float64(-338.5)], [np.float64(-339.5), np.float64(-337.5)], [np.float64(-336.5), np.float64(-335.5)], [np.float64(-336.5), np.float64(-335.5)], [np.float64(-334.5), np.float64(-333.5)], [np.float64(-333.5), -1], [np.float64(-333.5), np.float64(-332.5)], [np.float64(-332.5), np.float64(-320.5)], [np.float64(-332.5), np.float64(-324.5)], [np.float64(-325.5), np.float64(-324.5)], [np

In [24]:
import json

In [25]:
new_json_data = {}
for son in json_data:
    result = {}
    for i,v in enumerate(json_data[son]):
        result[i] = v
    new_json_data[son] = result

In [26]:
write_json("BettiZeroSegmentations", new_json_data)

TypeError: Object of type ndarray is not JSON serializable

In [27]:
import numpy as np
import json

def write_json(json_file_name, data):
    from pathlib import Path
    from collections.abc import Mapping, Sequence

    def convert(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, (Mapping, Sequence)) and not isinstance(obj, str):
            if isinstance(obj, Mapping):
                return {k: convert(v) for k, v in obj.items()}
            return [convert(v) for v in obj]
        return obj

    data = convert(data)

    output_path = Path(f"{json_file_name}.json")
    with output_path.open("w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


In [28]:
import numpy as np


write_json("betti_output", new_json_data)
